# Imports

In [1]:
using QuadGK
using Interpolations
using OrdinaryDiffEq

In [2]:
include("/home/jaimerz/PhD/LimberJack.jl/src/core.jl")

Cl (generic function with 1 method)

In [3]:
cosmoPar = CosmoPar(0.27, 0.049, 0.67, 0.96, 0.81)
cosmo = Cosmology(cosmoPar);

┌ Warning: extrapolate(itp, Linear()) is deprecated, use extrapolate(itp, Line()) instead
│   caller = #LinearInterpolation#52 at convenience-constructors.jl:9 [inlined]
└ @ Core /home/jaimerz/.julia/packages/Interpolations/3gTQB/src/convenience-constructors.jl:9
┌ Warning: extrapolate(itp, Linear()) is deprecated, use extrapolate(itp, Line()) instead
│   caller = #LinearInterpolation#51 at convenience-constructors.jl:7 [inlined]
└ @ Core /home/jaimerz/.julia/packages/Interpolations/3gTQB/src/convenience-constructors.jl:7
┌ Warning: extrapolate(itp, Linear()) is deprecated, use extrapolate(itp, Line()) instead
│   caller = #LinearInterpolation#52 at convenience-constructors.jl:9 [inlined]
└ @ Core /home/jaimerz/.julia/packages/Interpolations/3gTQB/src/convenience-constructors.jl:9


# Data

In [4]:
z_arr = [0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,  1]
b_arr = [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]
nz = [[0.0, 0.1, 0.2, 0.30000000000000004,
       0.4, 0.5, 0.6000000000000001, 0.7000000000000001,
       0.8, 0.9, 1.0],
      [0.0, 0.09549150281252627, 0.3454915028125263,
       0.6545084971874737, 0.9045084971874736, 1.0,
       0.9045084971874736, 0.6545084971874737, 0.34549150281252644,
       0.09549150281252634, 0]]
clustering_t = [0.00000000e+00, 4.48526950e-05, 1.71324904e-04, 3.43709458e-04,
        5.04105075e-04, 5.92283813e-04, 5.69713631e-04, 4.38454138e-04,
        2.46077204e-04, 7.22670075e-05, 1.03428840e-10]
lensing_t = [0.00000000e+00, 8.46720571e-06, 1.23677281e-05, 1.23488595e-05,
        9.80438814e-06, 6.34013396e-06, 3.25773350e-06, 1.22774325e-06,
        2.76337744e-07, 1.90522187e-08, 0.00000000e+00]
CMBlensing_t = [0.00000000e+00, 1.11161519e-05, 2.29070311e-05, 3.51531866e-05,
        4.76791214e-05, 6.03482340e-05, 7.30566167e-05, 8.57272238e-05,
        9.83037415e-05, 1.10746285e-04, 1.23027399e-04];
ell_arr = [ 31.5,  74. , 120. , 169.5, 223.5, 281.5, 344.];
lensing_lensing_cl_t = [6.96479316e-09, 2.62291354e-09, 1.39551291e-09, 8.99582785e-10,
       6.43044113e-10, 4.92082002e-10, 3.92524368e-10]
clust_lensing_cl_t = [2.66277949e-07, 1.14529794e-07, 6.13494474e-08, 3.84973684e-08,
       2.65390829e-08, 1.96776245e-08, 1.53408900e-08]
clust_clust_cl_t = [2.22573773e-05, 1.08124919e-05, 6.17595003e-06, 3.83844129e-06,
       2.61700894e-06, 1.88608858e-06, 1.43344030e-06]
CMBk_CMBk_cl_t = [2.13619136e-07, 1.84110942e-07, 1.39760222e-07, 1.05266694e-07,
       8.05684880e-08, 6.31640681e-08, 5.03579347e-08]
CMBlensing_clust_cl_t = [1.31477844e-06, 6.48418679e-07, 3.79843276e-07, 2.36573072e-07,
       1.62207584e-07, 1.16943272e-07, 8.86539366e-08]


7-element Array{Float64,1}:
 1.31477844e-6
 6.48418679e-7
 3.79843276e-7
 2.36573072e-7
 1.62207584e-7
 1.16943272e-7
 8.86539366e-8

# Compare Tracers

In [5]:
qg = clustering_kernel(cosmo, 2, nz)
clustering_jl = [qg(z, 1) for z in z_arr]

UndefVarError: [91mUndefVarError: clustering_kernel not defined[39m

In [6]:
clustering_jl./clustering_t

UndefVarError: [91mUndefVarError: clustering_jl not defined[39m

In [7]:
lensing_jl = [lensing_kernel(cosmo, z, nz) for z in z_arr]

11-element Array{Float64,1}:
 0.0
 7.1937899076368845e-6
 1.0586957153248261e-5
 1.0668050849629944e-5
 8.569027877940084e-6
 5.631462938389998e-6
 2.9639288564379416e-6
 1.1676961291062794e-6
 2.947709260635293e-7
 3.2248298952665044e-8
 0.0

In [ ]:
lensing_jl./lensing_t

In [ ]:
qCMBL = CMBlensing_kernel(cosmo)
CMBlensing_jl = [qCMBL(z, 1) for z in z_arr]

In [ ]:
CMBlensing_jl./CMBlensing_t

In [ ]:
CMBlensing_jl./CMBlensing_t

# Compare Cls

In [5]:
qgamma = shear_kernel(cosmo, nz)
qk = convergence_kernel(cosmo, nz)
qCMBk = CMBk_kernel(cosmo, 1100)
qg = clustering_kernel(cosmo, 2, nz)

qg (generic function with 1 method)

In [6]:
CMBlensing_kernel(cosmo, 3, 1100)

0.0005508201348330815

In [7]:
clust_clust_cl_jl = Cl(cosmo, ell_arr, qg, qg)

7-element Array{Float64,1}:
 2.5645297877069334e-5
 1.2045642353662797e-5
 6.232113544864445e-6
 3.555520313311439e-6
 2.1668026986192548e-6
 1.3956315676300789e-6
 9.358623522576653e-7

In [8]:
conv_conv_cl_jl = Cl(cosmo, ell_arr, qk, qk)

7-element Array{Float64,1}:
 6.155084546745092e-9
 1.912189774068467e-9
 8.27751707444321e-10
 4.2712575152953113e-10
 2.435239278464233e-10
 1.4953090982051393e-10
 9.665476688609746e-11

In [9]:
shear_shear_cl_jl = Cl(cosmo, ell_arr, qgamma, qgamma)

7-element Array{Float64,1}:
 6.143059961549997e-9
 1.911500696672405e-9
 8.276376920576224e-10
 4.2709619239829127e-10
 2.435142210088435e-10
 1.49527149160087e-10
 9.6653138056187e-11

In [10]:
CMBk_CMBk_cl_jl = Cl(cosmo, ell_arr, qCMBk, qCMBk)

7-element Array{Float64,1}:
 0.04308114654964907
 0.09641034548758436
 0.1520264923958245
 0.21004444686791438
 0.2714805452443718
 0.33549704497925237
 0.4023272734256988

In [11]:
clust_clust_cl_jl./clust_clust_cl_t

7-element Array{Float64,1}:
 1.1522156241233927
 1.1140486823081732
 1.0090939069441347
 0.9262927435087692
 0.8279691618551578
 0.739960775135004
 0.6528784995494165

In [12]:
conv_conv_cl_jl./lensing_lensing_cl_t

7-element Array{Float64,1}:
 0.8837426188181434
 0.729032713014424
 0.5931523108907112
 0.4748042744387679
 0.37870485542633886
 0.30387396655997573
 0.2462388956348755

In [13]:
shear_shear_cl_jl./lensing_lensing_cl_t

7-element Array{Float64,1}:
 0.8820161375115406
 0.7287699985232472
 0.5930706094704795
 0.47477141572722653
 0.37868976029152057
 0.30386632421497706
 0.2462347460073791

In [14]:
CMBk_CMBk_cl_jl./CMBk_CMBk_cl_t

7-element Array{Float64,1}:
 201672.6935439392
 523653.5343324915
      1.0877665348572822e6
      1.9953552152774397e6
      3.3695623684085e6
      5.31151737168196e6
      7.989352141276334e6